In [34]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from scipy.stats import multivariate_normal as mvn

In [35]:
model_alogp =  tf.keras.models.load_model("..//Base_Models//Model_BOTH")

2023-06-15 18:16:45.139736: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-15 18:16:45.139831: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-15 18:16:45.139884: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [36]:
# read small mol phase 3 features
small_mol_phase_3_features = pd.read_csv("..//Data/smiles_with_rdkit_with_small_phase_3_features.csv")

In [37]:
small_mol_phase_3_preds = model_alogp.predict([small_mol_phase_3_features["Smiles"].values, small_mol_phase_3_features.iloc[:,1:].values], batch_size = 1000)

2023-06-15 18:16:46.149077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-15 18:16:46.150117: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-15 18:16:46.150967: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 623ms/step


In [38]:
bottleneck_features = np.load("..//Data//small_mol_phase_3_features_for_both.npy")

In [39]:
bottleneck_features.shape

(959, 64)

In [40]:
smiles_features = bottleneck_features[:, :32]
rdkit_features = bottleneck_features[:, 32:]

In [41]:
smiles_weights = model_alogp.get_weights()[-2][:32, :]
rdkit_weights = model_alogp.get_weights()[-2][32:, :]

In [42]:
final_bias = model_alogp.get_weights()[-1].reshape(1,-1)

In [43]:
smiles_output = smiles_features@smiles_weights

In [44]:
rdkit_output = rdkit_features@rdkit_weights

In [45]:
(smiles_output + 0.5*final_bias) + (smiles_output + 0.5*final_bias)

array([[-0.3888892 , -0.03492932],
       [-0.9985395 , -0.9654727 ],
       [-0.18326166, -0.7627395 ],
       ...,
       [ 0.28031182,  0.49819827],
       [ 0.5116189 , -0.14718807],
       [-0.5595869 ,  1.3414242 ]], dtype=float32)

In [46]:
small_mol_phase_3_preds

array([[ 2.3886687e-03, -3.1844121e-01],
       [ 4.0491667e-01, -2.8562254e-01],
       [ 2.2429219e-01, -1.2114938e+00],
       ...,
       [-2.1914113e+00,  2.4811475e-01],
       [ 2.6189165e+00, -5.0469714e-01],
       [-5.5551714e-01,  1.1706109e+00]], dtype=float32)

In [47]:
# pd.read_csv("..//Data//y_train.csv")

In [48]:
y_train = pd.read_csv("..//Data//y_train.csv")
# y_train = pd.read_csv("..//Data/smiles_with_rdkit_with_small_phase_3_outputs.csv").iloc[:,1:]

In [49]:
std_targets = pickle.load(open('..//Data//target_scaler.pkl', 'rb'))

/home/statgrads/vpiyush2/.conda/envs/enkf/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [50]:
# std_targets = StandardScaler()

In [51]:
y_train = std_targets.transform(y_train)

In [52]:
np.cov(y_train.T)

array([[ 1.00000059, -0.30301738],
       [-0.30301738,  1.00000059]])

In [53]:
np.corrcoef(y_train.T)

array([[ 1.       , -0.3030172],
       [-0.3030172,  1.       ]])

In [54]:
# np.cov(std_targets.inverse_transform(y_train).T)

In [55]:
# weighted_outs = weighted_outs + final_bias

In [56]:
# np.save("..//Data//smiles_0.7_rdkit_0.3_signal.npy", weighted_outs)

In [57]:
covariance =  -0.3*0.2

In [58]:
R_t = np.array([[0.3,  covariance], [covariance, 0.3]])

In [59]:
R_t.shape

(2, 2)

In [60]:
R_t

array([[ 0.3 , -0.06],
       [-0.06,  0.3 ]])

In [61]:
# tf.linalg.cholesky(R_t).numpy()@tf.linalg.cholesky(R_t).numpy().T

In [62]:
# R_t

In [63]:
from sklearn.model_selection import train_test_split

In [64]:
def generate_data_copies(smiles_output, weight_smiles, R_t = np.cov(y_train.T), reps = 50):
    # weight_smiles = 0.8
    # weight_rdkit = 1-weight_smiles
    # weighted_outs = (weight_smiles*smiles_output + weight_rdkit*rdkit_output) 
    weighted_outs = (weight_smiles*smiles_output + weight_smiles*smiles_output) 
    weighted_outs = weighted_outs + final_bias
    error_mean = np.zeros((weighted_outs.shape[1],))
    error_cov = R_t
    catch = []
    for i in range(0, reps):
        mvn_error = mvn(error_mean, error_cov).rvs(weighted_outs.shape[0])
        weighted_outs_with_error = weighted_outs + mvn_error
        combo_y = np.hstack((weighted_outs, weighted_outs_with_error))
        x_train, x_valid, y_train, y_valid = train_test_split(bottleneck_features, combo_y, test_size = 0.25, shuffle = True, 
                                                     random_state = None)
        catch.append([x_train, x_valid, y_train, y_valid])
    return catch

In [65]:
catch = generate_data_copies(smiles_output, 0.5, R_t = R_t, reps = 50)

In [96]:
# catch[0][3]

In [66]:
with open("..//Data//changepoint_50_50_smiles.pickle", "wb") as f: 
    pickle.dump(catch, f)

In [98]:
# np.save("..//Data//smiles_0.8_rdkit_0.2_signal_plus_noise.npy", weighted_outs_with_error)